In [1]:
%matplotlib inline

import numpy as np
import scipy.linalg
import scipy.sparse.linalg
import matplotlib.pyplot as plt
import time
from numba import jit

In [2]:
@jit(nopython=True)
def build_pascal(lc,nc):
    cnkc = np.zeros((lc,nc),dtype=np.int64)
    for i in range(1,lc+1):
        cnkc[i-1,0] = 1
    for i in range(1,lc+1):
        for j in range(2,nc+1):
            cnkc[i-1,j-1] = 0
    for in1 in range(2,lc+1):
        cnkc[in1-1,1] = np.sum(cnkc[in1-2,0:2])
        if nc-1 > 1:
            for in2 in range(1,nc+1):
                cnkc[in1-1,in2-1] = np.sum(cnkc[in1-2,0:in2])
    return cnkc, cnkc[lc-1,nc-1]

In [3]:
lc = 7 # number of sites + 1
nc = 7 # number of atoms + 1
cnkc, jmax = build_pascal(lc,nc)
print(cnkc)
print(jmax)

[[  1   0   0   0   0   0   0]
 [  1   1   1   1   1   1   1]
 [  1   2   3   4   5   6   7]
 [  1   3   6  10  15  21  28]
 [  1   4  10  20  35  56  84]
 [  1   5  15  35  70 126 210]
 [  1   6  21  56 126 252 462]]
462


In [4]:
# Returns the position ind of the many body state bi
## original code, redundant if clauses
@jit(nopython=True)
def b2in_orig_redundant(bi,ind,cnkc,lc,nc): # basis to index
    ind[0] = 1 ## start from 1
#    ind[0] = 0 ## start from 0
    for ind_i in range(1,lc-1):
        for ind_N in range(0,bi[ind_i-1]+1):
            if bi[ind_i-1] - ind_N > 0:
                suma = 0
                for k in range(1,ind_i):
                    suma += bi[k-1]
                if lc - ind_i > 0 and nc - ind_N - suma > 0:
                    ind[0] += cnkc[lc-ind_i-1,nc-ind_N-suma-1]
    return 0

In [5]:
# Returns the position ind of the many body state bi
@jit(nopython=True)
def b2in_orig(bi,ind,cnkc,lc,nc): # basis to index
    ind[0] = 1 ## start from 1
#    ind[0] = 0 ## start from 0
    for ind_i in range(1,lc-1): # site loop
        for ind_N in range(0,bi[ind_i-1]): # atom loop
            suma = 0
            for k in range(1,ind_i):
                suma += bi[k-1]
            if nc - ind_N - suma > 0:
                ind[0] += cnkc[lc-ind_i-1,nc-ind_N-suma-1]
    return 0

In [6]:
# Returns the position ind of the many body state bi
@jit(nopython=True)
def b2in(bi,ind,cnkc,lc,nc): # basis to index
    ind[0] = 1 ## start from 1
    num = nc
    for ind_site in range(1,lc-1): # site loop
        numb = bi[ind_site-1]
        ind[0] += np.sum(cnkc[lc-1-ind_site,num-numb:num])
        #print(ind_site,num-1,numb,cnkc[lc-1-ind_site,num-numb:num])
        num -= numb
    return 0

In [7]:
lc = 7 # number of sites + 1
nc = 7 # number of atoms + 1
cnkc, jmax = build_pascal(lc,nc)

#bi = np.zeros(lc-1,dtype=np.int64)
ind = np.zeros(1,dtype=np.int64)
ind_orig = np.zeros(1,dtype=np.int64)

bi = np.array([2,1,1,0,2,0])
b2in_orig(bi,ind_orig,cnkc,lc,nc)
b2in(bi,ind,cnkc,lc,nc)
print(bi,ind_orig[0],ind[0])

bi = np.array([1,0,3,0,2,0])
b2in_orig(bi,ind_orig,cnkc,lc,nc)
b2in(bi,ind,cnkc,lc,nc)
print(bi,ind_orig[0],ind[0])

[2 1 1 0 2 0] 384 384
[1 0 3 0 2 0] 259 259


In [8]:
lc = 7 # number of sites + 1
nc = 7 # number of atoms + 1
cnkc, jmax = build_pascal(lc,nc)

#bi = np.zeros(lc-1,dtype=np.int64)
ind = np.zeros(1,dtype=np.int64)

bi = np.array([0,0,0,0,0,6])
b2in(bi,ind,cnkc,lc,nc)
print(bi,ind[0])

bi = np.array([1,1,1,1,1,1])
b2in(bi,ind,cnkc,lc,nc)
print(bi,ind[0])

bi = np.array([6,0,0,0,0,0])
b2in(bi,ind,cnkc,lc,nc)
print(bi,ind[0])

bi = np.array([1,0,3,0,2,0])
#bi = np.array([0,2,0,3,0,1])
b2in(bi,ind,cnkc,lc,nc)
print(bi,ind[0])

bi = np.array([2,1,1,0,2,0])
#bi = np.array([0,2,0,1,1,2])
b2in(bi,ind,cnkc,lc,nc)
print(bi,ind[0])

[0 0 0 0 0 6] 1
[1 1 1 1 1 1] 287
[6 0 0 0 0 0] 462
[1 0 3 0 2 0] 259
[2 1 1 0 2 0] 384


In [9]:
# Returns the position ind of the many body state bi_short
@jit(nopython=True)
def bshort2in(bis,ind,cnkc,lc,nc): # basis to index
    ind[0] = 1 ## start from 1
    num = 2
    for ind_atom in range(1,nc): # atom loop
        ind_site = bis[ind_atom-1]
        ind[0] += cnkc[ind_site-1,num-1]
        #print(ind_atom,ind_site,num,cnkc[ind_site-1,num-1],ind[0])
        num += 1
    return 0

In [10]:
lc = 7 # number of sites + 1
nc = 7 # number of atoms + 1
cnkc, jmax = build_pascal(lc,nc)

#bis = np.zeros(nc-1,dtype=np.int64)
ind = np.zeros(1,dtype=np.int64)

bis = np.array([2,2,4,5,6,6])
bshort2in(bis,ind,cnkc,lc,nc)
print(bis,ind[0])

[2 2 4 5 6 6] 384


In [11]:
# Returns the many body state bi at position ind
@jit(nopython=True)
def in2b(bi,ind,cnkc,lc,nc): # index to basis
    ind_i = ind[0] - 1 ## ind[0] runs from 1 to jmax=cnkc[ind_lc-1,ind_nc-1]
    bi[:] = 0
    ind_L = lc - 1
    ind_N = nc
    while ind_N > 1: # atom loop
        if ind_i >= cnkc[ind_L-1,ind_N-1]: # condition for site
            ind_i -= cnkc[ind_L-1,ind_N-1]
            bi[lc-ind_L-1] += 1
            ind_N -= 1
        else:
            ind_L -= 1
    return 0

In [12]:
lc = 7 # number of sites + 1
nc = 7 # number of atoms + 1
cnkc, jmax = build_pascal(lc,nc)

bi = np.zeros(lc-1,dtype=np.int64)
ind = np.zeros(1,dtype=np.int64)
ind[0] = 384
in2b(bi,ind,cnkc,lc,nc)
print(bi,ind[0])

[2 1 1 0 2 0] 384


In [13]:
# Returns the many body state bi_short at position ind
@jit(nopython=True)
def in2bshort(bis,ind,cnkc,lc,nc): # index to basis short
    ind_i = ind[0] - 1 ## ind[0] runs from 1 to jmax=cnkc[ind_lc-1,ind_nc-1]
    bis[:] = 0
    ind_L = lc - 1
    ind_N = nc
    while ind_N > 1: # atom loop
        if ind_i >= cnkc[ind_L-1,ind_N-1]: # condition for site
            ind_i -= cnkc[ind_L-1,ind_N-1]
            bis[ind_N-2] = ind_L
            ind_N -= 1
        else:
            ind_L -= 1
    return 0

In [14]:
lc = 7 # number of sites + 1
nc = 7 # number of atoms + 1
cnkc, jmax = build_pascal(lc,nc)

bis = np.zeros(nc-1,dtype=np.int64)
ind = np.zeros(1,dtype=np.int64)
ind[0] = 384
in2bshort(bis,ind,cnkc,lc,nc)
print(bis,ind[0])

[2 2 4 5 6 6] 384


In [15]:
## np.searchsorted is better?
@jit(nopython=True)
def binary_search_orig(s,list_s,ls,le):
    bmin = ls; bmax = le
    while True:
        b = bmin + (bmax-bmin)//2
        if s < list_s[b-1]:
            bmax = b - 1
        elif list_s[b-1] < s:
            bmin = b + 1
        else:
            bmin = b
            return b, bmin
        if bmin > bmax:
            b = -1
            return b, bmin
    return b, bmin

@jit(nopython=True)
def binary_search(s,list_s,ls,le,side="left"):
    return np.searchsorted(list_s[ls-1:le],s,side=side)+1

In [16]:
lc = 7 # number of sites + 1
nc = 7 # number of atoms + 1
cnkc, jmax = build_pascal(lc,nc)

print(cnkc[:,nc-1])
print()

for i in cnkc[:,nc-1]-2:
    b, bmin = binary_search_orig(i,cnkc[:,nc-1],1,lc)
    bmin2 = binary_search(i,cnkc[:,nc-1],1,lc)
    print(i,b,bmin,bmin2)
print()

for i in cnkc[:,nc-1]-1:
    b, bmin = binary_search_orig(i,cnkc[:,nc-1],1,lc)
    bmin2 = binary_search(i,cnkc[:,nc-1],1,lc)
    print(i,b,bmin,bmin2)
print()

for i in cnkc[:,nc-1]:
    b, bmin = binary_search_orig(i,cnkc[:,nc-1],1,lc)
    bmin2 = binary_search(i,cnkc[:,nc-1],1,lc)
    print(i,b,bmin,bmin2)
print()

for i in cnkc[:,nc-1]+1:
    b, bmin = binary_search_orig(i,cnkc[:,nc-1],1,lc)
    bmin2 = binary_search(i,cnkc[:,nc-1],1,lc)
    print(i,b,bmin,bmin2)
print()

for i in cnkc[:,nc-1]+2:
    b, bmin = binary_search_orig(i,cnkc[:,nc-1],1,lc)
    bmin2 = binary_search(i,cnkc[:,nc-1],1,lc)
    print(i,b,bmin,bmin2)
print()

[  0   1   7  28  84 210 462]

-2 -1 1 1
-1 -1 1 1
5 -1 3 3
26 -1 4 4
82 -1 5 5
208 -1 6 6
460 -1 7 7

-1 -1 1 1
0 1 1 1
6 -1 3 3
27 -1 4 4
83 -1 5 5
209 -1 6 6
461 -1 7 7

0 1 1 1
1 2 2 2
7 3 3 3
28 4 4 4
84 5 5 5
210 6 6 6
462 7 7 7

1 2 2 2
2 -1 3 3
8 -1 4 4
29 -1 5 5
85 -1 6 6
211 -1 7 7
463 -1 8 8

2 -1 3 3
3 -1 3 3
9 -1 4 4
30 -1 5 5
86 -1 6 6
212 -1 7 7
464 -1 8 8



In [17]:
# Returns the many body state bi_short at position ind
# using binary search
@jit(nopython=True)
def in2bshort_bs(bis,ind,cnkc,lc,nc): # index to basis short
    ind_i = ind[0] ## ind[0] runs from 1 to jmax=cnkc[ind_lc-1,ind_nc-1]
    bis[:] = 0
    ind_site = lc
    for ind_atom in range(nc,1,-1): # atom loop
#        icnkc, icnkcmin = binary_search_orig(ind_i,cnkc[:ind_site,ind_atom-1],1,ind_site)
        icnkcmin = binary_search(ind_i,cnkc[:ind_site,ind_atom-1],1,ind_site)
        #print(ind_atom,ind_i,icnkc,icnkcmin,cnkc[:ind_site,ind_atom-1],cnkc[icnkcmin-2,ind_atom-1])
        ind_i -= cnkc[icnkcmin-2,ind_atom-1]
        bis[ind_atom-2] = icnkcmin-1
        ind_site = icnkcmin
    return 0

In [18]:
lc = 7 # number of sites + 1
nc = 7 # number of atoms + 1
cnkc, jmax = build_pascal(lc,nc)

bis = np.zeros(nc-1,dtype=np.int64)
ind = np.zeros(1,dtype=np.int64)

ind[0] = 384
in2bshort_bs(bis,ind,cnkc,lc,nc)
print(bis,ind[0])
print()

ind[0] = 259
in2bshort_bs(bis,ind,cnkc,lc,nc)
print(bis,ind[0])
print()

ind[0] = 1
in2bshort_bs(bis,ind,cnkc,lc,nc)
print(bis,ind[0])
print()

ind[0] = jmax
in2bshort_bs(bis,ind,cnkc,lc,nc)
print(bis,ind[0])
print()

[2 2 4 5 6 6] 384

[2 2 4 4 4 6] 259

[1 1 1 1 1 1] 1

[6 6 6 6 6 6] 462



In [19]:
lc = 7 # number of sites + 1
nc = 7 # number of atoms + 1
cnkc, jmax = build_pascal(lc,nc)

bi = np.zeros(lc-1,dtype=np.int64)
bis = np.zeros(nc-1,dtype=np.int64)
bis_bs = np.zeros(nc-1,dtype=np.int64)
ind = np.zeros(1,dtype=np.int64)
ind_bi = np.zeros(1,dtype=np.int64)
ind_bis = np.zeros(1,dtype=np.int64)
ind_bis_bs = np.zeros(1,dtype=np.int64)

#for i in range(1,jmax+1):
for i in list(range(1,9))+[259]+[384]+list(range(jmax+1-8,jmax+1)):
    ind[0] = i
    in2b(bi,ind,cnkc,lc,nc)
    in2bshort(bis,ind,cnkc,lc,nc)
    in2bshort_bs(bis_bs,ind,cnkc,lc,nc)
    b2in(bi,ind_bi,cnkc,lc,nc)
    bshort2in(bis,ind_bis,cnkc,lc,nc)
    bshort2in(bis_bs,ind_bis_bs,cnkc,lc,nc)
    print(ind[0],bi,ind_bi[0]," ",ind[0],bis,ind_bis[0]," ",ind[0],bis_bs,ind_bis_bs[0])

1 [0 0 0 0 0 6] 1   1 [1 1 1 1 1 1] 1   1 [1 1 1 1 1 1] 1
2 [0 0 0 0 1 5] 2   2 [1 1 1 1 1 2] 2   2 [1 1 1 1 1 2] 2
3 [0 0 0 0 2 4] 3   3 [1 1 1 1 2 2] 3   3 [1 1 1 1 2 2] 3
4 [0 0 0 0 3 3] 4   4 [1 1 1 2 2 2] 4   4 [1 1 1 2 2 2] 4
5 [0 0 0 0 4 2] 5   5 [1 1 2 2 2 2] 5   5 [1 1 2 2 2 2] 5
6 [0 0 0 0 5 1] 6   6 [1 2 2 2 2 2] 6   6 [1 2 2 2 2 2] 6
7 [0 0 0 0 6 0] 7   7 [2 2 2 2 2 2] 7   7 [2 2 2 2 2 2] 7
8 [0 0 0 1 0 5] 8   8 [1 1 1 1 1 3] 8   8 [1 1 1 1 1 3] 8
259 [1 0 3 0 2 0] 259   259 [2 2 4 4 4 6] 259   259 [2 2 4 4 4 6] 259
384 [2 1 1 0 2 0] 384   384 [2 2 4 5 6 6] 384   384 [2 2 4 5 6 6] 384
455 [4 1 1 0 0 0] 455   455 [4 5 6 6 6 6] 455   455 [4 5 6 6 6 6] 455
456 [4 2 0 0 0 0] 456   456 [5 5 6 6 6 6] 456   456 [5 5 6 6 6 6] 456
457 [5 0 0 0 0 1] 457   457 [1 6 6 6 6 6] 457   457 [1 6 6 6 6 6] 457
458 [5 0 0 0 1 0] 458   458 [2 6 6 6 6 6] 458   458 [2 6 6 6 6 6] 458
459 [5 0 0 1 0 0] 459   459 [3 6 6 6 6 6] 459   459 [3 6 6 6 6 6] 459
460 [5 0 1 0 0 0] 460   460 [4 6 6 6 6 6] 460 

In [20]:
#lc = 33 # number of sites + 1
lc = 25 # number of sites + 1
nc = 4 # number of atoms + 1
cnkc, jmax = build_pascal(lc,nc)

bi = np.zeros(lc-1,dtype=np.int64)
bis = np.zeros(nc-1,dtype=np.int64)
bis_bs = np.zeros(nc-1,dtype=np.int64)
ind = np.zeros(1,dtype=np.int64)
ind_bi = np.zeros(1,dtype=np.int64)
ind_bis = np.zeros(1,dtype=np.int64)
ind_bis_bs = np.zeros(1,dtype=np.int64)

#for i in range(1,jmax+1):
for i in list(range(1,9))+list(range(jmax+1-8,jmax+1)):
    ind[0] = i
    in2b(bi,ind,cnkc,lc,nc)
    in2bshort(bis,ind,cnkc,lc,nc)
    in2bshort_bs(bis_bs,ind,cnkc,lc,nc)
    b2in(bi,ind_bi,cnkc,lc,nc)
    bshort2in(bis,ind_bis,cnkc,lc,nc)
    bshort2in(bis_bs,ind_bis_bs,cnkc,lc,nc)
    print(ind[0],bi,ind_bi[0]," ",ind[0],bis,ind_bis[0]," ",ind[0],bis_bs,ind_bis_bs[0])

1 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 3] 1   1 [1 1 1] 1   1 [1 1 1] 1
2 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 2] 2   2 [1 1 2] 2   2 [1 1 2] 2
3 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 1] 3   3 [1 2 2] 3   3 [1 2 2] 3
4 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 3 0] 4   4 [2 2 2] 4   4 [2 2 2] 4
5 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 2] 5   5 [1 1 3] 5   5 [1 1 3] 5
6 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1] 6   6 [1 2 3] 6   6 [1 2 3] 6
7 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 2 0] 7   7 [2 2 3] 7   7 [2 2 3] 7
8 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 1] 8   8 [1 3 3] 8   8 [1 3 3] 8
2593 [2 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] 2593   2593 [17 24 24] 2593   2593 [17 24 24] 2593
2594 [2 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] 2594   2594 [18 24 24] 2594   2594 [18 24 24] 2594
2595 [2 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] 2595   2595 [19 24 24] 2595   2595 [19 24 24] 2595
2596 [2 0 0 0 1 0 0 0 0 0 

In [21]:
@jit(nopython=True)
def calc_ni(bis,nc):
    return [np.sum(bis==i) for i in range(nc)]

@jit(nopython=True)
def calc_ni2(bis,nc):
    return [np.sum(bis==i)**2 for i in range(nc)]

In [22]:
lc = 7 # number of sites + 1
nc = 7 # number of atoms + 1
cnkc, jmax = build_pascal(lc,nc)

bis = np.zeros(nc-1,dtype=np.int64)
ind = np.zeros(1,dtype=np.int64)

ind[0] = 384
in2bshort_bs(bis,ind,cnkc,lc,nc)
print(bis,ind[0],calc_ni(bis,nc),calc_ni2(bis,nc))
print()

ind[0] = 259
in2bshort_bs(bis,ind,cnkc,lc,nc)
print(bis,ind[0],calc_ni(bis,nc),calc_ni2(bis,nc))
print()

ind[0] = 1
in2bshort_bs(bis,ind,cnkc,lc,nc)
print(bis,ind[0],calc_ni(bis,nc),calc_ni2(bis,nc))
print()

ind[0] = jmax
in2bshort_bs(bis,ind,cnkc,lc,nc)
print(bis,ind[0],calc_ni(bis,nc),calc_ni2(bis,nc))
print()

[2 2 4 5 6 6] 384 [0, 0, 2, 0, 1, 1, 2] [0, 0, 4, 0, 1, 1, 4]

[2 2 4 4 4 6] 259 [0, 0, 2, 0, 3, 0, 1] [0, 0, 4, 0, 9, 0, 1]

[1 1 1 1 1 1] 1 [0, 6, 0, 0, 0, 0, 0] [0, 36, 0, 0, 0, 0, 0]

[6 6 6 6 6 6] 462 [0, 0, 0, 0, 0, 0, 6] [0, 0, 0, 0, 0, 0, 36]



In [23]:
## binary search not efficient for small nmax?
## this binary search code "binary_search_orig" has bug, no good with duplication
#@jit(nopython=True)
#def calc_aiadj(bis2,bis,i,j,nmax): # assume i<j
#    _, x = binary_search_orig(i,bis,1,nmax)
#    x += np.sum(bis==i) - 1
#    _, y = binary_search_orig(j,bis,1,nmax)
#    y += np.sum(bis==j) - 1
#    bis2[0:x-1] = bis[0:x-1]
#    bis2[x-1:y-1] = bis[x:y]
#    bis2[y-1] = j
#    bis2[y:nmax] = bis[y:nmax]
#    return 0

#@jit(nopython=True)
#def calc_aiadj(bis2,bis,i,j,nmax): # assume i<j
#    _, x = binary_search_orig(i,bis,1,nmax)
#    x += np.sum(bis==i) - 1
#    _, y = binary_search_orig(j,bis,1,nmax)
#    y += np.sum(bis==j) - 1
#    print()
#    print("#",i,binary_search_orig(i,bis,1,nmax),x,y)
#    x2 = binary_search(i,bis,1,nmax,side="left")
#    y2 = binary_search(j,bis,1,nmax,side="left")
#    x3 = binary_search(i,bis,1,nmax,side="right")
#    y3 = binary_search(j,bis,1,nmax,side="right")
#    print("#",i,x2,y2)
#    print("#",i,x3,y3)
#    print("#",i,x2 + np.sum(bis==i) - 1,y2 + np.sum(bis==j) - 1)
#    bis2[0:x-1] = bis[0:x-1]
#    bis2[x-1:y-1] = bis[x:y]
#    bis2[y-1] = j
#    bis2[y:nmax] = bis[y:nmax]
#    return 0

@jit(nopython=True)
def calc_aiadj(bis2,bis,i,j,nmax): # assume i<j
    x = binary_search(i,bis,1,nmax) + np.sum(bis==i) - 1
    y = binary_search(j,bis,1,nmax) + np.sum(bis==j) - 1
    bis2[0:x-1] = bis[0:x-1]
    bis2[x-1:y-1] = bis[x:y]
    bis2[y-1] = j
    bis2[y:nmax] = bis[y:nmax]
    return 0

In [24]:
lc = 7 # number of sites + 1
nc = 7 # number of atoms + 1
cnkc, jmax = build_pascal(lc,nc)

bis = np.zeros(nc-1,dtype=np.int64)
bis2 = np.zeros(nc-1,dtype=np.int64)
ind = np.zeros(1,dtype=np.int64)
ind2 = np.zeros(1,dtype=np.int64)

ind[0] = 384
#ind[0] = 259
#ind[0] = 1
#ind[0] = 2
#ind[0] = 3
#ind[0] = jmax
in2bshort_bs(bis,ind,cnkc,lc,nc)
print(bis,ind[0])
for i in range(1,lc):
    for j in range(i+1,lc):
        if i in bis:
            calc_aiadj(bis2,bis,i,j,nc-1)
            bshort2in(bis2,ind2,cnkc,lc,nc)
            coeff = np.sqrt(np.sum(bis==i)*(np.sum(bis==j)+1)) # \sqrt(n_i(nj+1))
            print(i,j,bis2,ind2[0],coeff)
print()

[2 2 4 5 6 6] 384
2 3 [2 3 4 5 6 6] 386 1.4142135623730951
2 4 [2 4 4 5 6 6] 389 2.0
2 5 [2 4 5 5 6 6] 399 2.0
2 6 [2 4 5 6 6 6] 434 2.449489742783178
4 5 [2 2 5 5 6 6] 394 1.4142135623730951
4 6 [2 2 5 6 6 6] 429 1.7320508075688772
5 6 [2 2 4 6 6 6] 419 1.7320508075688772



In [25]:
lc = 4 # number of sites + 1
nc = 3 # number of atoms + 1
cnkc, jmax = build_pascal(lc,nc)

bis = np.zeros(nc-1,dtype=np.int64)
bis2 = np.zeros(nc-1,dtype=np.int64)
ind = np.zeros(1,dtype=np.int64)
ind2 = np.zeros(1,dtype=np.int64)

for id in range(1,jmax+1):
    ind[0] = id
    in2bshort_bs(bis,ind,cnkc,lc,nc)
    print(bis,ind[0])
    for i in range(1,lc):
        for j in range(i+1,lc):
            if i in bis:
                calc_aiadj(bis2,bis,i,j,nc-1)
                bshort2in(bis2,ind2,cnkc,lc,nc)
                coeff = np.sqrt(np.sum(bis==i)*(np.sum(bis==j)+1)) # \sqrt(n_i(nj+1))
                print(i,j,bis2,ind2[0],coeff)
    print()
print()

[1 1] 1
1 2 [1 2] 2 1.4142135623730951
1 3 [1 3] 4 1.4142135623730951

[1 2] 2
1 2 [2 2] 3 1.4142135623730951
1 3 [2 3] 5 1.0
2 3 [1 3] 4 1.0

[2 2] 3
2 3 [2 3] 5 1.4142135623730951

[1 3] 4
1 2 [2 3] 5 1.0
1 3 [3 3] 6 1.4142135623730951

[2 3] 5
2 3 [3 3] 6 1.4142135623730951

[3 3] 6




In [26]:
## binary search not efficient for small nmax?
## this binary search code "binary_search_orig" has bug, no good with duplication
#@jit(nopython=True)
#def calc_adiaj(bis2,bis,i,j,nmax): # assume i<j
#    _, x = binary_search_orig(i,bis,1,nmax)
#    x += np.sum(bis==i)
#    _, y = binary_search_orig(j,bis,1,nmax)
#    bis2[0:x-1] = bis[0:x-1]
#    bis2[x-1] = i
#    bis2[x:y] = bis[x-1:y-1]
#    bis2[y:nmax] = bis[y:nmax]
#    return 0

@jit(nopython=True)
def calc_adiaj(bis2,bis,i,j,nmax): # assume i<j
    x = binary_search(i,bis,1,nmax) + np.sum(bis==i)
    y = binary_search(j,bis,1,nmax)
    bis2[0:x-1] = bis[0:x-1]
    bis2[x-1] = i
    bis2[x:y] = bis[x-1:y-1]
    bis2[y:nmax] = bis[y:nmax]
    return 0

In [27]:
lc = 7 # number of sites + 1
nc = 7 # number of atoms + 1
cnkc, jmax = build_pascal(lc,nc)

bis = np.zeros(nc-1,dtype=np.int64)
bis2 = np.zeros(nc-1,dtype=np.int64)
ind = np.zeros(1,dtype=np.int64)
ind2 = np.zeros(1,dtype=np.int64)

ind[0] = 384
#ind[0] = 259
#ind[0] = 1
#ind[0] = 2
#ind[0] = 3
#ind[0] = jmax
in2bshort_bs(bis,ind,cnkc,lc,nc)
print(bis,ind[0])
for i in range(1,lc):
    for j in range(i+1,lc):
        if j in bis:
            calc_adiaj(bis2,bis,i,j,nc-1)
            bshort2in(bis2,ind2,cnkc,lc,nc)
            coeff = np.sqrt((np.sum(bis==i)+1)*np.sum(bis==j)) # \sqrt((n_i+1)nj)
            print(i,j,bis2,ind2[0],coeff)
print()

[2 2 4 5 6 6] 384
1 2 [1 2 4 5 6 6] 383 1.4142135623730951
1 4 [1 2 2 5 6 6] 374 1.0
1 5 [1 2 2 4 6 6] 354 1.0
1 6 [1 2 2 4 5 6] 284 1.4142135623730951
2 4 [2 2 2 5 6 6] 375 1.7320508075688772
2 5 [2 2 2 4 6 6] 355 1.7320508075688772
2 6 [2 2 2 4 5 6] 285 2.449489742783178
3 4 [2 2 3 5 6 6] 378 1.0
3 5 [2 2 3 4 6 6] 358 1.0
3 6 [2 2 3 4 5 6] 288 1.4142135623730951
4 5 [2 2 4 4 6 6] 364 1.4142135623730951
4 6 [2 2 4 4 5 6] 294 2.0
5 6 [2 2 4 5 5 6] 314 2.0



In [28]:
lc = 4 # number of sites + 1
nc = 3 # number of atoms + 1
cnkc, jmax = build_pascal(lc,nc)

bis = np.zeros(nc-1,dtype=np.int64)
bis2 = np.zeros(nc-1,dtype=np.int64)
ind = np.zeros(1,dtype=np.int64)
ind2 = np.zeros(1,dtype=np.int64)

for id in range(1,jmax+1):
    ind[0] = id
    in2bshort_bs(bis,ind,cnkc,lc,nc)
    print(bis,ind[0])
    for i in range(1,lc):
        for j in range(i+1,lc):
            if j in bis:
                calc_adiaj(bis2,bis,i,j,nc-1)
                bshort2in(bis2,ind2,cnkc,lc,nc)
                coeff = np.sqrt((np.sum(bis==i)+1)*np.sum(bis==j)) # \sqrt((n_i+1)nj)
                print(i,j,bis2,ind2[0],coeff)
    print()
print()

[1 1] 1

[1 2] 2
1 2 [1 1] 1 1.4142135623730951

[2 2] 3
1 2 [1 2] 2 1.4142135623730951

[1 3] 4
1 3 [1 1] 1 1.4142135623730951
2 3 [1 2] 2 1.0

[2 3] 5
1 2 [1 3] 4 1.0
1 3 [1 2] 2 1.0
2 3 [2 2] 3 1.4142135623730951

[3 3] 6
1 3 [1 3] 4 1.4142135623730951
2 3 [2 3] 5 1.4142135623730951




In [29]:
@jit(nopython=True)
def make_full_hamiltonian(lv,Ham,cnkc,lc,nc,no_U,val_U,no_J,pair_J,val_J):
    bis = np.zeros(nc-1,dtype=np.int64)
    bis2 = np.zeros(nc-1,dtype=np.int64)
    ind = np.zeros(1,dtype=np.int64)
    ind2 = np.zeros(1,dtype=np.int64)
    for i in range(1,lv+1): # state loop
        ind[0] = i
        in2bshort_bs(bis,ind,cnkc,lc,nc)
        for j in range(1,no_U+1): # site loop
            nj = np.sum(bis==j)
            Ham[i-1,i-1] += 0.5 * val_U[j-1] * nj * (nj-1.0)
        for j in range(1,no_J+1): # bond loop
            f1 = pair_J[0,j-1] in bis
            f2 = pair_J[1,j-1] in bis
            if f1:
                calc_aiadj(bis2,bis,pair_J[0,j-1],pair_J[1,j-1],nc-1)
                bshort2in(bis2,ind2,cnkc,lc,nc)
                Ham[i-1,ind2[0]-1] -= val_J[j-1] * \
                    np.sqrt(np.sum(bis==pair_J[0,j-1]) * (np.sum(bis==pair_J[1,j-1])+1))
#                print("### 1 aiadj",bis2,bis,pair_J[0,j-1],pair_J[1,j-1])
            if f2:
                calc_adiaj(bis2,bis,pair_J[0,j-1],pair_J[1,j-1],nc-1)
                bshort2in(bis2,ind2,cnkc,lc,nc)
                Ham[i-1,ind2[0]-1] -= val_J[j-1] * \
                    np.sqrt((np.sum(bis==pair_J[0,j-1])+1) * np.sum(bis==pair_J[1,j-1]))
#                print("### 2 adiaj",bis2,bis,pair_J[0,j-1],pair_J[1,j-1])
    return 0

In [30]:
@jit(nopython=True)
def make_parameters_1d(lc,U,J):
    no_U = lc - 1
    val_U = U * np.ones(no_U,dtype=np.float64)
    no_J = lc - 1
    pair_J = np.zeros((2,no_J),dtype=np.int64)
    val_J = J * np.ones(no_J,dtype=np.float64)
    for i in range(no_J):
        pair_J[0,i] = i%no_J+1
        pair_J[1,i] = (i+1)%no_J+1
        if pair_J[0,i] > pair_J[1,i]: # assume i<j for pair (i,j)
            tmp = pair_J[0,i]
            pair_J[0,i] = pair_J[1,i]
            pair_J[1,i] = tmp
    return no_U, val_U, no_J, pair_J, val_J

In [31]:
#@jit(nopython=True)
def calculate_1d_full_diag(lc,nc,U,J):
    start = time.time()
#    lc = 21 # number of sites + 1
#    nc = 4 # number of atoms + 1
    cnkc, jmax = build_pascal(lc,nc)
    print("# cnkc",cnkc)
    print("# total Hilbert space size",jmax)
    end = time.time()
    print("## time build_pascal",end-start)
    print()

    start = time.time()
#    U = 10.0
#    J = 1.0
    no_U, val_U, no_J, pair_J, val_J = make_parameters_1d(lc,U,J)
    print("# no_U",no_U)
    print("# val_U",val_U)
    print("# no_J",no_J)
    print("# pair_J",pair_J)
    print("# val_J",val_J)
    end = time.time()
    print("## time make_parameters_1d",end-start)
    print()

    start = time.time()
    Ham = np.zeros((jmax,jmax),dtype=np.float64)
    make_full_hamiltonian(jmax,Ham,cnkc,lc,nc,no_U,val_U,no_J,pair_J,val_J)
#    print(Ham)
#    print(Ham.T-Ham)
#    print("# \sum |Ham-Ham.T|",np.sum(np.abs(Ham.T-Ham)))
#    for i in range(jmax):
#        for j in range(jmax):
#            if np.abs(Ham[i,j]) > 1e-6:
#                print(i,j,Ham[i,j])
    end = time.time()
    print("## time make Hamiltonian",end-start)
    print()

    start = time.time()
    ene, vec = scipy.linalg.eigh(Ham)
    idx = np.argsort(ene)
    ene = ene[idx]
    vec = vec[:,idx]
    print("# ene",*ene[0:np.min([jmax,5])].flatten())
    #print("# vec",vec[:,0:np.min([jmax,5])])
    end = time.time()
    print("## time diagonalization",end-start)
    print()

In [32]:
lc = 31 # number of sites + 1
nc = 4 # number of atoms + 1
U = 10.0
J = 1.0
calculate_1d_full_diag(lc,nc,U,J)

# cnkc [[   1    0    0    0]
 [   1    1    1    1]
 [   1    2    3    4]
 [   1    3    6   10]
 [   1    4   10   20]
 [   1    5   15   35]
 [   1    6   21   56]
 [   1    7   28   84]
 [   1    8   36  120]
 [   1    9   45  165]
 [   1   10   55  220]
 [   1   11   66  286]
 [   1   12   78  364]
 [   1   13   91  455]
 [   1   14  105  560]
 [   1   15  120  680]
 [   1   16  136  816]
 [   1   17  153  969]
 [   1   18  171 1140]
 [   1   19  190 1330]
 [   1   20  210 1540]
 [   1   21  231 1771]
 [   1   22  253 2024]
 [   1   23  276 2300]
 [   1   24  300 2600]
 [   1   25  325 2925]
 [   1   26  351 3276]
 [   1   27  378 3654]
 [   1   28  406 4060]
 [   1   29  435 4495]
 [   1   30  465 4960]]
# total Hilbert space size 4960
## time build_pascal 0.0006768703460693359

# no_U 30
# val_U [10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10.
 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10.]
# no_J 30
# pair_J [[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14

In [33]:
def ham_to_vec_wave_vector(lv,cnkc,lc,nc,no_U,val_U,no_J,pair_J,val_J):
    @jit(nopython=True)
    def get_vec(v1,v0,bis,bis2,ind,ind2): ## v0: new output, v1: old input
        for i in range(1,lv+1): # state loop
#            v0[i-1] = 0.0 + 0.0j
            v0[i-1] = 0.0
            ind[0] = i
            in2bshort_bs(bis,ind,cnkc,lc,nc)
            for j in range(1,no_U+1): # site loop
                nj = np.sum(bis==j)
                v0[i-1] += 0.5 * val_U[j-1] * nj * (nj-1.0) * v1[i-1]
            for j in range(1,no_J+1): # bond loop
                f1 = pair_J[0,j-1] in bis
                f2 = pair_J[1,j-1] in bis
                if f1:
                    calc_aiadj(bis2,bis,pair_J[0,j-1],pair_J[1,j-1],nc-1)
                    bshort2in(bis2,ind2,cnkc,lc,nc)
                    v0[i-1] -= val_J[j-1] * \
                        np.sqrt(np.sum(bis==pair_J[0,j-1]) * (np.sum(bis==pair_J[1,j-1])+1)) * v1[ind2[0]-1]
                if f2:
                    calc_adiaj(bis2,bis,pair_J[0,j-1],pair_J[1,j-1],nc-1)
                    bshort2in(bis2,ind2,cnkc,lc,nc)
                    v0[i-1] -= val_J[j-1] * \
                        np.sqrt((np.sum(bis==pair_J[0,j-1])+1) * np.sum(bis==pair_J[1,j-1])) * v1[ind2[0]-1]
        return v0
    return get_vec

In [34]:
#@jit(nopython=True)
def calculate_1d_linearoperator(lc,nc,U,J):
    start = time.time()
#    lc = 21 # number of sites + 1
#    nc = 4 # number of atoms + 1
    cnkc, jmax = build_pascal(lc,nc)
    print("# cnkc",cnkc)
    print("# total Hilbert space size",jmax)
    end = time.time()
    print("## time build_pascal",end-start)
    print()

    start = time.time()
#    U = 10.0
#    J = 1.0
    no_U, val_U, no_J, pair_J, val_J = make_parameters_1d(lc,U,J)
    print("# no_U",no_U)
    print("# val_U",val_U)
    print("# no_J",no_J)
    print("# pair_J",pair_J)
    print("# val_J",val_J)
    end = time.time()
    print("## time make_parameters_1d",end-start)
    print()

    start = time.time()
    get_vec = ham_to_vec_wave_vector(jmax,cnkc,lc,nc,no_U,val_U,no_J,pair_J,val_J)
    bis = np.zeros(nc-1,dtype=np.int64)
    bis2 = np.zeros(nc-1,dtype=np.int64)
    ind = np.zeros(1,dtype=np.int64)
    ind2 = np.zeros(1,dtype=np.int64)
    #v0 = np.zeros(jmax,dtype=np.complex128)
    v0 = np.zeros(jmax,dtype=np.float64)
    Ham = scipy.sparse.linalg.LinearOperator((jmax,jmax),matvec=lambda v1: get_vec(v1,v0,bis,bis2,ind,ind2))
    end = time.time()
    print("## time make Hamiltonian",end-start)
    print()

    start = time.time()
    ene, vec = scipy.sparse.linalg.eigsh(Ham,which="SA",k=np.min([5,jmax-1]))
    idx = ene.argsort()
    ene = ene[idx]
    vec = vec[:,idx]
    print("# ene",*ene[0:np.min([jmax,5])].flatten())
    #print("# vec",vec[:,0:np.min([jmax,5])])
    end = time.time()
    print("## time diagonalization",end-start)
    print()

In [35]:
lc = 31 # number of sites + 1
nc = 4 # number of atoms + 1
U = 10.0
J = 1.0
calculate_1d_linearoperator(lc,nc,U,J)

# cnkc [[   1    0    0    0]
 [   1    1    1    1]
 [   1    2    3    4]
 [   1    3    6   10]
 [   1    4   10   20]
 [   1    5   15   35]
 [   1    6   21   56]
 [   1    7   28   84]
 [   1    8   36  120]
 [   1    9   45  165]
 [   1   10   55  220]
 [   1   11   66  286]
 [   1   12   78  364]
 [   1   13   91  455]
 [   1   14  105  560]
 [   1   15  120  680]
 [   1   16  136  816]
 [   1   17  153  969]
 [   1   18  171 1140]
 [   1   19  190 1330]
 [   1   20  210 1540]
 [   1   21  231 1771]
 [   1   22  253 2024]
 [   1   23  276 2300]
 [   1   24  300 2600]
 [   1   25  325 2925]
 [   1   26  351 3276]
 [   1   27  378 3654]
 [   1   28  406 4060]
 [   1   29  435 4495]
 [   1   30  465 4960]]
# total Hilbert space size 4960
## time build_pascal 0.0005371570587158203

# no_U 30
# val_U [10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10.
 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10.]
# no_J 30
# pair_J [[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14

In [36]:
@jit(nopython=True)
def make_sparse_hamiltonian_child(lv,cnkc,lc,nc,no_U,val_U,no_J,pair_J,val_J):
    bis = np.zeros(nc-1,dtype=np.int64)
    bis2 = np.zeros(nc-1,dtype=np.int64)
    ind = np.zeros(1,dtype=np.int64)
    ind2 = np.zeros(1,dtype=np.int64)
    lstki = np.array([i for k in range(2*no_J+1) for i in range(lv)],dtype=np.int64)
    lstloc = np.zeros((2*no_J+1)*lv,dtype=np.int64)
#    lstele = np.zeros((2*no_J+1)*lv,dtype=np.complex128)
    lstele = np.zeros((2*no_J+1)*lv,dtype=np.float64)
    for i in range(1,lv+1): # state loop
        ind[0] = i
        in2bshort_bs(bis,ind,cnkc,lc,nc)
        lstloc[2*no_J*lv+(i-1)] = i-1 # diagonal localtion
        for j in range(1,no_U+1): # site loop
            nj = np.sum(bis==j)
            lstele[2*no_J*lv+(i-1)] += 0.5 * val_U[j-1] * nj * (nj-1.0) # diagonal element
#            Ham[i-1,i-1] += 0.5 * val_U[j-1] * nj * (nj-1.0)
        for j in range(1,no_J+1): # bond loop
            f1 = pair_J[0,j-1] in bis
            f2 = pair_J[1,j-1] in bis
            if f1:
                calc_aiadj(bis2,bis,pair_J[0,j-1],pair_J[1,j-1],nc-1)
                bshort2in(bis2,ind2,cnkc,lc,nc)
                lstele[(j-1)*lv+(i-1)] -= val_J[j-1] * \
                    np.sqrt(np.sum(bis==pair_J[0,j-1]) * (np.sum(bis==pair_J[1,j-1])+1)) # offdiag element
                lstloc[(j-1)*lv+(i-1)] = ind2[0]-1 # offdiag localtion
#                Ham[i-1,ind2[0]-1] -= val_J[j-1] * \
#                    np.sqrt(np.sum(bis==pair_J[0,j-1]) * (np.sum(bis==pair_J[1,j-1])+1))
#                print("### 1 aiadj",bis2,bis,pair_J[0,j-1],pair_J[1,j-1])
            if f2:
                calc_adiaj(bis2,bis,pair_J[0,j-1],pair_J[1,j-1],nc-1)
                bshort2in(bis2,ind2,cnkc,lc,nc)
                lstele[((j-1)+no_J)*lv+(i-1)] -= val_J[j-1] * \
                    np.sqrt((np.sum(bis==pair_J[0,j-1])+1) * np.sum(bis==pair_J[1,j-1])) # offdiag element
                lstloc[((j-1)+no_J)*lv+(i-1)] = ind2[0]-1 # offdiag localtion
#                Ham[i-1,ind2[0]-1] -= val_J[j-1] * \
#                    np.sqrt((np.sum(bis==pair_J[0,j-1])+1) * np.sum(bis==pair_J[1,j-1]))
#                print("### 2 adiaj",bis2,bis,pair_J[0,j-1],pair_J[1,j-1])
    return lstele, lstki, lstloc

In [37]:
def make_sparse_hamiltonian(lv,lstele,lstki,lstloc):
#    return scipy.sparse.csr_matrix((lstele,(lstki,lstloc)),shape=(lv,lv),dtype=np.complex128)
    return scipy.sparse.csr_matrix((lstele,(lstki,lstloc)),shape=(lv,lv),dtype=np.float64)

In [38]:
#@jit(nopython=True)
def calculate_1d_sparse(lc,nc,U,J):
    start = time.time()
#    lc = 21 # number of sites + 1
#    nc = 4 # number of atoms + 1
    cnkc, jmax = build_pascal(lc,nc)
    print("# cnkc",cnkc)
    print("# total Hilbert space size",jmax)
    end = time.time()
    print("## time build_pascal",end-start)
    print()

    start = time.time()
#    U = 10.0
#    J = 1.0
    no_U, val_U, no_J, pair_J, val_J = make_parameters_1d(lc,U,J)
    print("# no_U",no_U)
    print("# val_U",val_U)
    print("# no_J",no_J)
    print("# pair_J",pair_J)
    print("# val_J",val_J)
    end = time.time()
    print("## time make_parameters_1d",end-start)
    print()

    start = time.time()
    lstele, lstki, lstloc = make_sparse_hamiltonian_child(jmax,cnkc,lc,nc,no_U,val_U,no_J,pair_J,val_J)
    Ham = make_sparse_hamiltonian(jmax,lstele,lstki,lstloc)
#    print(Ham)
    end = time.time()
    print("## time make Hamiltonian",end-start)
    print()

    start = time.time()
    ene, vec = scipy.sparse.linalg.eigsh(Ham,which="SA",k=np.min([5,jmax-1]))
    idx = ene.argsort()
    ene = ene[idx]
    vec = vec[:,idx]
    print("# ene",*ene[0:np.min([jmax,5])].flatten())
    #print("# vec",vec[:,0:np.min([jmax,5])])
    end = time.time()
    print("## time diagonalization",end-start)
    print()

In [39]:
lc = 31 # number of sites + 1
nc = 4 # number of atoms + 1
U = 10.0
J = 1.0
calculate_1d_sparse(lc,nc,U,J)

# cnkc [[   1    0    0    0]
 [   1    1    1    1]
 [   1    2    3    4]
 [   1    3    6   10]
 [   1    4   10   20]
 [   1    5   15   35]
 [   1    6   21   56]
 [   1    7   28   84]
 [   1    8   36  120]
 [   1    9   45  165]
 [   1   10   55  220]
 [   1   11   66  286]
 [   1   12   78  364]
 [   1   13   91  455]
 [   1   14  105  560]
 [   1   15  120  680]
 [   1   16  136  816]
 [   1   17  153  969]
 [   1   18  171 1140]
 [   1   19  190 1330]
 [   1   20  210 1540]
 [   1   21  231 1771]
 [   1   22  253 2024]
 [   1   23  276 2300]
 [   1   24  300 2600]
 [   1   25  325 2925]
 [   1   26  351 3276]
 [   1   27  378 3654]
 [   1   28  406 4060]
 [   1   29  435 4495]
 [   1   30  465 4960]]
# total Hilbert space size 4960
## time build_pascal 0.0008041858673095703

# no_U 30
# val_U [10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10.
 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10. 10.]
# no_J 30
# pair_J [[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14